<a href="https://colab.research.google.com/github/Do-cc/cc/blob/master/bi-predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
def create_dataset(dataset, look_back):
    '''
    对数据进行处理
    '''
    dataX, dataY = [], []
    for i in range (len (dataset) - (look_back * 2) -1, len (dataset) - look_back - 1):
        a = dataset[i:(i + look_back), :]
        dataX.append (a)
        dataY.append (dataset[i + look_back, :])
    TrainX = np.array (dataX)
    Train_Y = np.array (dataY)

    return TrainX, Train_Y

# 多维归一化  返回数据和最大最小值
def NormalizeMult(data):
    # normalize 用于反归一化
    data = np.array (data)
    normalize = np.arange (2 * data.shape[1], dtype='float64')

    normalize = normalize.reshape (data.shape[1], 2)
    print (normalize.shape)
    for i in range (0, data.shape[1]):
        # 第i列
        list = data[:, i]
        listlow, listhigh = np.percentile (list, [0, 100])
        # print(i)
        normalize[i, 0] = listlow
        normalize[i, 1] = listhigh
        delta = listhigh - listlow
        if delta != 0:
            # 第j行
            for j in range (0, data.shape[0]):
                data[j, i] = (data[j, i] - listlow) / delta
    # np.save("./normalize.npy",normalize)
    return data, normalize


# 多维反归一化
def FNormalizeMult(data, normalize):
    data = np.array (data)
    for i in range (0, data.shape[1]):
        listlow = normalize[i, 0]
        listhigh = normalize[i, 1]
        delta = listhigh - listlow
        if delta != 0:
            # 第j行
            for j in range (0, data.shape[0]):
                data[j, i] = data[j, i] * delta + listlow

    return data

model = load_model ('bi-attention.h5')
data = pd.read_excel ('imfs1.xls')
data = data.drop (['number'], axis=1)
# data = data.drop (['number', 'LR corr', 'LR loc', 'LS corr', 'LS loc'], axis=1)
data = np.array(data)
INPUT_DIMS = 7
TIME_STEPS = 24
data, normalize = NormalizeMult (data)
test_X, _ = create_dataset(data, TIME_STEPS)
print(test_X.shape)

predictions = model.predict (test_X)
predictions = FNormalizeMult(predictions, normalize)
# MM = MinMaxScaler()
#　predictions = MM.fit.inverse_transform (predictions)
actual = FNormalizeMult(data[-24:, [0]], normalize)

# plt.plot(FNormalizeMult(data[:, [0]],normalize))

plt.plot(predictions)
plt.plot(actual)
plt.xlabel ('timestamp', fontsize=12)
plt.ylabel ('load', fontsize=12)
plt.show ()


OSError: ignored

# 新段落